In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
import requests as r
import pandas_datareader as pdr
from dotenv import load_dotenv
import os
load_dotenv()
token = os.getenv("ALPHA")
from time import sleep

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-24 2021-12-29


In [16]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
#         data = TiingoExtractor.extract(ticker,start,end)
#         data["ticker"] = ticker
#         market.store("prices",data)
        test = pdr.yahoo.daily.YahooDailyReader(symbols=ticker
                                                , start=start
                                                , end=end
                                                , retry_count=3, pause=0.1, session=None, adjust_price=True)
        panda_reader_data = test.read()
        panda_reader_data["ticker"] = ticker
        panda_reader_data.reset_index(inplace=True)
        market.store("pdr_prices",panda_reader_data)
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'
        stuff = r.get(url,params={"apikey":token,"symbol":ticker,"outputsize":"compact"})
        data = stuff.json()
        example = pd.DataFrame.from_records(data["Time Series (Daily)"]).T
        for col in example.columns:
            example.rename(columns={col:col.split(".")[1].strip()},inplace=True)
        example.reset_index(inplace=True)
        example.rename(columns={"index":"date"},inplace=True)
        example["ticker"] = ticker
        market.store("alpha_prices",example)
        sleep(12)
    except Exception as e:
        print(str(e))
market.disconnect()

 15%|██████                                   | 75/505 [16:36<1:08:13,  9.52s/it]

'Date'


 15%|██████▋                                    | 78/505 [17:03<58:06,  8.17s/it]

'Date'


 28%|███████████▉                              | 143/505 [31:21<57:09,  9.48s/it]

...

 73%|█████████████████████████████▏          | 368/505 [1:22:42<24:46, 10.85s/it]

'Date'


 88%|███████████████████████████████████▏    | 444/505 [1:39:33<09:35,  9.44s/it]

'Date'


100%|████████████████████████████████████████| 505/505 [1:53:40<00:00, 13.51s/it]


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|█████▋                                     | 67/505 [00:04<00:33, 13.00it/s]

'date'
'date'


 16%|██████▉                                    | 82/505 [00:05<00:23, 18.27it/s]

'date'
'date'


 20%|████████▍                                 | 102/505 [00:07<00:22, 18.03it/s]

...

'date'


 84%|███████████████████████████████████▏      | 423/505 [00:30<00:05, 16.34it/s]

'date'


100%|██████████████████████████████████████████| 505/505 [00:35<00:00, 14.15it/s]

'date'
'date'
'date'
'date'


In [17]:
period = "day"
a = pd.DataFrame(analysis)
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker
80,-0.077290,0.053154,0.086445,0.027975,0.027975,CPB
59,0.058805,0.051773,0.037290,0.025892,0.025892,BLL
27,0.225380,-0.143649,0.138821,0.020363,0.020363,AAL
456,0.412870,0.025455,0.106819,0.018734,0.018734,VLO
5,-0.254793,-0.151025,0.163932,0.018640,0.018640,ATVI
378,0.348273,0.223428,0.127157,0.016251,0.016251,PHM
146,0.095266,0.093567,0.024822,0.016156,0.016156,DG
136,0.023238,-0.126708,0.181927,0.016154,0.016154,DAL
77,0.174126,0.235524,0.136545,0.015792,0.015792,CHRW
472,0.015707,0.015707,0.015707,0.015707,0.015707,DIS


In [18]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

KeyError: 'GICS Sector'